In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 1)

In [27]:
df = pd.read_csv('C:/ind_sl_t20i_1st_2022_02_24.csv')

In [28]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))

In [29]:
df['Playing_xi'] = 1

In [30]:
df.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,Playing_xi
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,False,False,False,True,True,False,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,False,True,False,False,True,False,1
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,False,False,True,False,True,False,1
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,False,True,False,False,False,True,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,False,True,False,False,True,False,1


In [38]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

mo = pulp.LpProblem('Points maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Playing_xi']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

In [39]:
mo.solve()

1

In [40]:
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

In [41]:
fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [42]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,Playing_xi,pickup_status
0,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,False,True,False,False,True,False,1,1
1,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,False,True,False,False,False,True,1,1
2,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,False,True,False,False,True,False,1,1
3,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,False,False,False,True,True,False,1,1
4,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1,True,False,False,False,True,False,1,1
5,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1,True,False,False,False,True,False,1,1
6,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1,True,False,False,False,False,True,1,1
7,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,False,False,True,False,True,False,1,1
8,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1,False,False,True,False,True,False,1,1
9,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1,False,False,True,False,False,True,1,1


In [43]:
sum(fdf.Credits)

97.0

In [46]:
np.mean(fdf.Credits)

8.818181818181818

In [47]:
np.mean(fdf.Points)

59.36363636363637